<b>The Problem Statement</b>:

Write a Python program to solve the Travelling Salesman problem using Branch and Bound approach.

Imagine a salesman who needs to visit a set of cities and return to his starting point while minimizing the total distance traveled. Let's consider a small set of cities with their pairwise distances:

City A to City B: 10 miles

City A to City C: 15 miles

City A to City D: 20 miles

City B to City C: 35 miles

City B to City D: 25 miles

City C to City D: 30 miles

The goal of the TSP is to find the shortest possible route that visits each city exactly once and returns to the starting city.

## The Code

### Imports

In [1]:
import heapq
import sys

### Definition of the Heuristic Function

In [2]:
# Define heuristic functions
def calculate_path_cost(graph, path):
    """
    Calculate the total cost of a given path in the graph.

    Args:
        graph (list of lists): The distance matrix representing the distances between cities.
        path (list of ints): The path (sequence of city indices) for which to calculate the cost.

    Returns:
        int: The total cost of the path.
    """
    # Initialize the cost to 0
    cost = 0

    # Loop through the cities in the path
    for i in range(len(path) - 1):
        # Add the distance between the current city and the next city to the cost
        cost += graph[path[i]][path[i + 1]]

    # Return the total cost of the path
    return cost

### Lower Bound Calculation

In [3]:
def calculate_lower_bound(graph, visited_cities, current_city):
    """
    Calculate the lower bound for the current state using the minimum outgoing edge and the minimum incoming edge.

    Args:
        graph (list of lists): The distance matrix representing the distances between cities.
        visited_cities (set of ints): The set of cities that have been visited so far.
        current_city (int): The index of the current city.

    Returns:
        int: The calculated lower bound for the current state.
    """
    # Initialize the minimum outgoing edge cost to a large value
    min_outgoing_edge = sys.maxsize

    # Find the minimum outgoing edge from the current city to unvisited cities
    for city in range(len(graph)):
        if city not in visited_cities and graph[current_city][city] < min_outgoing_edge:
            min_outgoing_edge = graph[current_city][city]

    # Initialize the minimum incoming edge cost to a large value
    min_incoming_edge = sys.maxsize

    # Find the minimum cost to reach unvisited cities from visited cities
    for city in range(len(graph)):
        if city not in visited_cities and city != current_city:
            # Initialize the minimum cost to the current city to a large value
            min_cost_to_city = sys.maxsize

            # Find the minimum cost to reach the current city from visited cities
            for visited_city in visited_cities:
                min_cost_to_city = min(min_cost_to_city, graph[visited_city][city])

            # Update the minimum incoming edge cost
            min_incoming_edge = min(min_incoming_edge, min_cost_to_city)

    # Calculate the total lower bound by summing the minimum outgoing and incoming edge costs
    lower_bound = min_outgoing_edge + min_incoming_edge

    return lower_bound

### Priority Queue Creation

In [4]:
def initialize_priority_queue(num_cities):
   """
    Initialize the priority queue with the initial state of the search.
    
    Args:
        num_cities (int): The total number of cities in the TSP instance.
    
    Returns:
        list: A priority queue containing the initial state.
    """
   initial_state = (0, {0}, [0])
   priority_queue = [initial_state]
   return priority_queue

### Define the Branch and Bound algorithm

In [9]:
def tsp_branch_and_bound(graph):
    # Get the number of cities
    num_cities = len(graph)
    
    # Initialize variables to track the best solution
    best_path = None
    best_cost = sys.maxsize
    
    # Initialize the priority queue with the initial state
    priority_queue = initialize_priority_queue(num_cities)

    # Explore the state space using a while loop
    while priority_queue:
        # Pop the current state from the priority queue
        current_state = heapq.heappop(priority_queue)
        current_cost, visited_cities, path = current_state
        # If all cities are visited, check if it's a complete tour
        if len(visited_cities) == num_cities:
            # Add the cost to return to the starting city
            current_cost += graph[path[-1]][path[0]]
            
            # Update the best solution if the current cost is better
            if current_cost < best_cost:
                print(priority_queue)
                best_cost = current_cost
                best_path = path[:]
        else:
            # Explore child states by considering unvisited cities
            current_city = path[-1]
            for city in range(num_cities):
                if city not in visited_cities:
                    child_path = path + [city]
                    child_cost = current_cost + graph[current_city][city]
                    
                    # Calculate a lower bound for the child state
                    lower_bound = calculate_lower_bound(graph, visited_cities, current_city)

                    # If the child state has the potential to improve the solution, add it to the priority queue
                    if child_cost + lower_bound < best_cost:
                        child_state = (child_cost, visited_cities | {city}, child_path)
                        heapq.heappush(priority_queue, child_state)

    # Return the best path and cost found
    return best_path, best_cost

### Main function to solve the problem

In [10]:
# Example usage
if __name__ == "__main__":
    graph = [
        [0, 10, 15, 20],
        [10, 0, 35, 25],
        [15, 35, 0, 30],
        [20, 25, 30, 0]
    ]

    best_path, best_cost = tsp_branch_and_bound(graph)
    print("Best TSP Path:", best_path)
    print("Best TSP Cost:", best_cost)

[(70, {0, 1, 2, 3}, [0, 2, 3, 1]), (75, {0, 1, 2, 3}, [0, 1, 2, 3]), (75, {0, 1, 2, 3}, [0, 2, 1, 3]), (85, {0, 1, 2, 3}, [0, 3, 2, 1]), (80, {0, 1, 2, 3}, [0, 3, 1, 2])]
Best TSP Path: [0, 1, 3, 2]
Best TSP Cost: 80


---

```
BranchAndBoundTSP(graph):
    num_cities = number of cities in the graph
    best_path = None
    best_cost = infinity
    priority_queue = initialize_priority_queue(num_cities)

    while priority_queue is not empty:
        current_state = pop state with lowest cost from priority_queue
        current_cost = cost of current_state
        visited_cities = cities visited in current_state
        current_path = path in current_state

        if all cities are visited:
            current_cost += cost of returning to the starting city
            if current_cost < best_cost:
                best_cost = current_cost
                best_path = current_path
        else:
            current_city = last city in current_path
            for city in unvisited cities:
                child_path = current_path + [city]
                child_cost = current_cost + cost from current_city to city
                lower_bound = calculate_lower_bound(graph, visited_cities, current_city)

                if child_cost + lower_bound < best_cost:
                    child_state = (child_cost, visited_cities + [city], child_path)
                    insert child_state into priority_queue

    return best_path, best_cost

```